<a href="https://colab.research.google.com/github/apapaioannou92/BA/blob/HW1/aap2204_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IEOR 4650  Business Analytics (Fall 2019)**

**Homework 1 [Total points: 60]**

** Due: 11:59 PM, September 28**

This first homework consists of Python programming practices and the applications to data clearning and linear regression. 

The steps to finish in the homework:

*   Step 1: Make a copy of the file to your LionDrive.
*   Step 2: Work with the copy (an ipynb file).
*    <font color='red'>**Step 3: Rename the copy to your_uni_hw1.ipynb (For example: yd4501_hw1.ipynb).**</font>  
*   Step 4: Submit the copy on CourseWorks.

Before you submit your assignment, make sure to re-run your code from the beginning. (You can do so by first clicking Runtime/Reset All Runtimes and then clicking Runtime/Run all)


**All the questions in this homework requires coding**

In [0]:
#This imports all the packages you need for the homework
#Please run this first


!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#import modules

import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt



## Exercise 1 [Total: 22 Points]

In this exercise, we will work on some exercises that practices our data manipulation skills.




<div>
<img src="https://http2.mlstatic.com/poster-pokemon-full-hd-42x29cm-frete-gratis-D_NQ_NP_752705-MLB25053489227_092016-F.jpg" width="250"/>
</div>

## Setting
Pokemon is a very popular video game, in which the trainer catches pokemon and use them for battling. Each pokemon has its name and its pokedex. A Pokemon belongs to at least 1 and at most 2 Pokemon types.  There are different stats to measure the combat power of a pokemon, which include HP, Attack, Defense, Special Attack Strength, Special Defensive Strength, and Speed. Pokemon are usually released in batches. Thus, they have different generations. In addition, a pokemon can either be a regular pokemon and legendary pokemon. 



|Variable| Description|
|---|---|
|PokeDex |PokeDex index number|
| Name | Name of the Pokemon  |  
| Type 1 | Type of pokemon  |  
| Type 2 | Other Type of Pokemon  |  
| HP |  Hit Points  |  
| Attack |  Attack Strength  |  
| Defense |  Defensive Strength  |  
| Sp. Atk |  Special Attack Strength  |  
| Sp. Def |  Special Defensive Strength  |  
| Speed | Speed  |  
| Generation | Number of generation  |  
| Legendary |  True if Legendary Pokemon False if not | "

In [0]:
#Load the dataset 
link="https://drive.google.com/open?id=1wX36WLNOPLPVU7BCA-J9bBhGZPwuFqol"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('myfile.csv')  
Pokemon = pd.read_csv('myfile.csv')
Pokemon.head()

,PokeDex,Name,Type_1,Type_2,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Charmander,Fire,***,39,52,43,60,50,65,1,False
4,5,Charmeleon,Fire,***,58,64,58,80,65,80,1,False


### Q1 [2 points]
* Find the number of unique pokemon in the table. [hint: you want to make sure the same pokemon name does not appear twice in the table]

**Make sure to print out the result**

In [0]:
Pokemon["Name"].nunique()

721

### Q2. [4 points]

* Add one additional column named "Total" to the table 
> The value of Total for each pokemon is equal to the sum of all the stats numbers for each pokemon (HP+Attack+Defense+Sp. Atk+Sp. Def+ Speed) and is the measure of total combat power of a pokemon.
* drop the stats that are used to compute "Total" from the table

**Make sure to print out the result for the last step**

In [0]:
Pokemon = Pokemon.assign(Total=Pokemon['HP']+Pokemon['Attack']+Pokemon['Defense']+Pokemon['Sp_Atk']+Pokemon['Sp_Def']+Pokemon['Speed'])
Pokemon.drop(['HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed'], axis=1, inplace=True)
Pokemon.head()

,PokeDex,Name,Type_1,Type_2,Generation,Legendary,Total
0,1,Bulbasaur,Grass,Poison,1,False,318
1,2,Ivysaur,Grass,Poison,1,False,405
2,3,Venusaur,Grass,Poison,1,False,525
3,4,Charmander,Fire,***,1,False,309
4,5,Charmeleon,Fire,***,1,False,405


### Q3 [4 points]

* Create a sub-table "Pokemon_sub" that includes only the pokemon name and the Pokedex for all the **Legendary**  pokemon from **Generation 1** and **Generation 2**. 

**Make sure to print out the content of the sub-table**

In [0]:
filter_price = (Pokemon["Generation"]==1) | (Pokemon["Generation"]==2) & (Pokemon['Legendary']=="True")
Pokemon_sub = Pokemon.loc[filter_price, ['Name', 'PokeDex']]
Pokemon_sub.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Name,PokeDex
0,Bulbasaur,1
1,Ivysaur,2
2,Venusaur,3
3,Charmander,4
4,Charmeleon,5


###Q4 [5 points]

A pokemon might belong to only one type or two types. If a pokemons belongs to only one type, "Type_2" field will be missing that Pokemon

We call a pokemon that belongs to only one type a "single-type" pokemon and a pokemon that belongs to two types a "double-type" pokemon. For example, Snorlax is a "single-type" pokemon and belongs to Normal type. For another example, Bulbasaur is a "double-type" pokemon and belongs to both Poison type and Grass type. 	  

* Convert the missing values to a format that can be recognized by Pandas.
* Report for each pokemon type, how many "single-type" pokemon exists.

**Make sure to print out the result for the last step**

In [0]:
Pokemon['Type_1'].value_counts()

Water       105
Normal       93
Grass        66
Bug          63
Psychic      47
Fire         47
Rock         41
Electric     36
Ground       30
Poison       28
Dark         28
Fighting     25
Dragon       24
Ice          23
Ghost        23
Steel        22
Fairy        17
Flying        3
Name: Type_1, dtype: int64

In [0]:
Pokemon2 = Pokemon.replace("***", np.nan)
Pokemon2.head()
Pokemon2["Type_1"].nunique() # returns 18 unique entries for Type 1
Pokemon2["Type_1"].value_counts()

Water       105
Normal       93
Grass        66
Bug          63
Psychic      47
Fire         47
Rock         41
Electric     36
Ground       30
Poison       28
Dark         28
Fighting     25
Dragon       24
Ice          23
Ghost        23
Steel        22
Fairy        17
Flying        3
Name: Type_1, dtype: int64

In [0]:
Pokemon3 = Pokemon.replace("***", np.nan)
Pokemon3.head()
filter_type = pd.isnull(Pokemon3["Type_2"])
Pokemon3 = Pokemon3.loc[filter_type, ['PokeDex', 'Name', 'Type_1', 'Type_2', 'Generation', 'Legendary',
       'Total']]
Pokemon3["Type_1"].nunique() # returns 18 unique entries for Type 1
Pokemon3["Type_1"].value_counts()

Normal      60
Water       57
Grass       33
Psychic     32
Fire        28
Electric    26
Fighting    20
Bug         17
Poison      15
Fairy       15
Ground      13
Ice         12
Dragon      11
Dark         9
Rock         9
Ghost        9
Steel        4
Flying       1
Name: Type_1, dtype: int64

In [0]:
#replace and delete missing values 
import numpy as np
Pokemon4 = Pokemon.replace("***", np.nan)
Pokemon4 = Pokemon4.dropna(subset = ['Type_2'])
Pokemon4["Type_1"].value_counts()

Water       48
Bug         46
Normal      33
Grass       33
Rock        32
Dark        19
Fire        19
Steel       18
Ground      17
Psychic     15
Ghost       14
Dragon      13
Poison      13
Ice         11
Electric    10
Fighting     5
Flying       2
Fairy        2
Name: Type_1, dtype: int64

In [1]:
import numpy as np
Pokemon5 = Pokemon.replace("***", np.nan)
Pokemon5 = Pokemon5[pd.isnull(Pokemon5['Type_2'])]
Pokemon5["Type_1"].value_counts()

NameError: ignored

In [0]:
Pokemon6 = Pokemon.replace("***", np.nan)
Pokemon7 = Pokemon6.dropna(subset=['Type_2'])
Pokemon8 = Pokemon6.drop(Pokemon7.index)
Pokemon8["Type_1"].value_counts()

Normal      60
Water       57
Grass       33
Psychic     32
Fire        28
Electric    26
Fighting    20
Bug         17
Poison      15
Fairy       15
Ground      13
Ice         12
Dragon      11
Dark         9
Rock         9
Ghost        9
Steel        4
Flying       1
Name: Type_1, dtype: int64

In [0]:
Pokemon_sub.head()

,Name,PokeDex
0,Bulbasaur,1
1,Ivysaur,2
2,Venusaur,3
3,Charmander,4
4,Charmeleon,5


###Q5 [7 points]

It is important to have the ability of learning programming by yourself. Let's use some adequate methods to find the pokemon with the highest total combat power in each type.  

In this table, you should have 3 columns

(1) column 1: Type name

(2) column 2: name of the most power pokemon for this pokemon type

(3) column 3: total combat power for this pokemon

[hint: You might find the following four methods to be relevant. However, there are multiple ways to achieve the goal. You do not need to use them in your answer. 

> `concat` method

> `sort_values` method

> `groupby` method

> `first` method

In [0]:
Pokemon_new = Pokemon[['Type_1', 'Type_2', 'Name', 'Total']].copy()
Pokemon_new['Type'] = Pokemon_new['Type_1'] +' & '+ Pokemon_new['Type_2']
Pokemon_new.drop(columns = ["Type_1", "Type_2"], inplace=True)
Pokemon_new = Pokemon_new[['Type', 'Name', 'Total']]
Pokemon_new = Pokemon_new.sort_values("Total", ascending=False).groupby('Type').first()
Pokemon_new.head(60)

,Name,Total
Type,,
Bug & ***,Pinsir,500
Bug & Electric,Galvantula,472
Bug & Fighting,Heracross,500
Bug & Fire,Volcarona,550
Bug & Flying,Yanmega,515
Bug & Ghost,Shedinja,236
Bug & Grass,Leavanny,500
Bug & Ground,Nincada,266
Bug & Poison,Scolipede,485


## Exercise 2 [Total: 38 points]



###Setting
In this exercise, we will work with a dataset which explores the store sales.

Before doing the assignment, please study this dataset here https://www.kaggle.com/c/rossmann-store-sales/data


In [0]:
#Load the dataset
link="https://drive.google.com/open?id=1Q6J0q4tlWJ7TajX6hMSvZ8tK3CyDD8QY"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Sales.csv')  
Sales = pd.read_csv('Sales.csv')
Sales.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


*You might see a warning when importing the data. This is because StateHoliday has both values in string and numbers. No worries this for now.*

### Q1 [7 points]

Run the following model

$$Sales=\beta_0+\beta_1 Customers + \epsilon$$

> (1) Print out the summary output

> (2) Based on regression result, answer the following questions

>> Interpret  $\hat{\beta_0}$ and $\hat{\beta_1}$ based on the regression output. 

>> Reason whether there is a negative correlation between the number of customers and Sales

>> Report the 99% confidence interval for $\beta_1$

In [0]:
model = ols(formula="Sales~Customers",data=Sales).fit()
print(model.summary(), model.conf_int(0.01), sep='\n \n \n')

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.801
Method:                 Least Squares   F-statistic:                 4.082e+06
Date:                Tue, 24 Sep 2019   Prob (F-statistic):               0.00
Time:                        15:36:46   Log-Likelihood:            -9.0214e+06
No. Observations:             1017209   AIC:                         1.804e+07
Df Residuals:                 1017207   BIC:                         1.804e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1077.7365      2.883    373.869      0.0

### $\beta_{1}$ interpretation
If you increase the Customers independent variable by 1 unit without changing other factors the expected value for the Sales target variable will increase ON AVERAGE by 7.4171 (+- 0.004)

### $\beta_{0}$ interpretation
If the Customers independent variable is equal to 0 the expected value for the Sales target variable will be on average equal to 1077.7365 (+- 2.883)

Also:
The intercept (often labeled the constant-here $\beta_{0}$) is the expected mean value of Sales when all independent variables, in our case Customers=0. 

If X (array of independent variables-in our case just Customers) sometimes equals 0, the intercept is simply the expected mean value of Y (target variable-in our case, Sales) at that value. If X never equals 0, then the intercept has no intrinsic meaning.

### Reason whether there is a negative correlation between the number of customers and Sales
From the regression table we observe that there is not a negative correlation with the Sales target variable and the Customers predictor variable. The reason why the independent variable, Customers, does not have a negative correlation with the Sales target variable is that, as the number of customers increases the dollar amount of Sales obtained is going to be higher.

### 99% confidence interval for $\beta_{1}$
The 99% confidence interval of $\beta_{1}$ is the interval [7.407605, 7.426518].

###Q2. [7 points]

* For the model above, use a Python attribute we discussed in class to retrieve the values for $\hat{\beta_0}$ and $\hat{\beta_1}$.

* Based on these two values, give the prediction for $\widehat{Sales}$ when number of customers=110.

* Find out the average of the actual Sales price for all the records that have customers=110.

In [0]:
#predicted values for beta

model.params

# print the intercept
b0_coeff = model.params[0]
intercept = b0_coeff

# print the first coefficient of b1
b1_coeff = model.params[1]
b1_coeff

print(b0_coeff, b1_coeff)

Sales_110 = b0_coeff + b1_coeff * 110

print(Sales_110)

print(Sales['Sales'].loc[Sales['Customers'] == 110].mean())

1077.7365031848192 7.417061572866315
1893.613276200114
908.1428571428571


###Q3 [8 points]

One person argues the following:

* He found from the data that we should expect 0 Sales when we have 0 customers
* Since $\beta_0$ is the expected Sales price, when the number of customers is equal to 0, it should be set equals to 0.  

> Based on data, support that his first argument is true.

> Provide an argument on why the second point might be problematic. 

You will find the following articles useful:
[Article 1](https://dynamicecology.wordpress.com/2017/04/13/dont-force-your-regression-through-zero-just-because-you-know-the-true-intercept-has-to-be-zero/)

In [0]:
# makes sense and it can be verifies by predicting the following quantity
print(Sales['Sales'].loc[Sales['Customers'] == 0].mean())

0.0


The form of the function of our regression models is ordinarily chosen for convenience and/or because it fits the observed data reasonably well, or for other reasons. In those cases we should not be forcing our intercept through the origin. By forcing our intercept through the origin in such cases is a way of papering over misspecification of your regression. For example, if our linear regression model provides a reasonable fit over range of X values we observed, but with a nonsensical estimated intercept, that’s our linear regression’s way of telling you that the true shape of F(X) is nonlinear near the origin, outside the range of X values you observed. Forcing the intercept through the origin treats the (most obvious) symptom (the nonsensical estimated intercept) but not the disease (misspecification).

###Q4. [5 points] 

Now, run the following model:

$Sales=\beta_0+\beta_1 Customers +\beta_2 Open + \epsilon$

> Fit the model and print out the summary table.

> Interpret $\hat{\beta_0}$ , $\hat{\beta_1}$ and $\hat{\beta_2}$. 


In [0]:
model = ols(formula="Sales~Customers+Open",data=Sales).fit()
print(model.summary(), sep='\n \n \n')

# For 0 customers and for Open = 0, which means for stores that are closed, the expected value for the target variable named Sales is on average -2.7e-11 ± 3.859, which is 
# practically 0.
# For 1 unit increase of the variable named Customers, the expected value for the target variable named Sales will on average increase by 6.3720 ± 0.004 monetary (USD?) units.
# For 1 unit increase of the variable named Open (so from closed stores to open stores), the expected value for the target variable named Sales will on average increase by 
# 2095.4121 ± 5.380 monetary (USD?) units.

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.826
Model:                            OLS   Adj. R-squared:                  0.826
Method:                 Least Squares   F-statistic:                 2.421e+06
Date:                Tue, 24 Sep 2019   Prob (F-statistic):               0.00
Time:                        15:36:52   Log-Likelihood:            -8.9507e+06
No. Observations:             1017209   AIC:                         1.790e+07
Df Residuals:                 1017206   BIC:                         1.790e+07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -2.7e-11      3.859     -7e-12      1.0

###Q5. [4 points]

Based on regression, prove that Q4 is equivalent to regressing "Sales" on "Customers" using only observations with stores that are open. 

In [0]:
filter_ = (Sales["Open"] == 1)
Sales = Sales[filter_]
# Sales["Open"].nunique()
# Sales["Open"].value_counts()
model = ols(formula="Sales~Customers",data=Sales).fit()
print(model.summary(), sep='\n \n \n')
print('\n')
print('We observe that the beta coefficients in both models are exactly the same and there are very slight differences only in their respective standard \n deviation values. Thus the model of Q4 is equivalent to regressing "Sales" on "Customers" using only observations with stores that are open.')

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.678
Method:                 Least Squares   F-statistic:                 1.780e+06
Date:                Tue, 24 Sep 2019   Prob (F-statistic):               0.00
Time:                        15:48:04   Log-Likelihood:            -7.5086e+06
No. Observations:              844392   AIC:                         1.502e+07
Df Residuals:                  844390   BIC:                         1.502e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2095.4121      4.116    509.149      0.0

###Q6. [7 points]
Store table gives the information of each store. 

* Merge Sales table with Store table
* Run the following regression model

$$Sales=\beta_0+\beta_1 Customers +  \beta_2 Open + \beta_3 Competition Distance+\epsilon$$

* Based on the regression result, interpret $\widehat{\beta_3}$. Is this consistent with your expectation? Why?

In [0]:
link="https://drive.google.com/open?id=1g6URzUJnhCLOtg0a9TTHC18KYcqwxvoc"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('myfile_store.csv')  
Store = pd.read_csv('myfile_store.csv')
# Store.head(5)

Sales = pd.read_csv('Sales.csv')
# Sales.head()
Sales_Store_merged = Sales.merge(Store, on = "Store") # since Store is the only 1 common column in both dfs
Sales_Store_merged.head()

model = ols(formula="Sales~Customers+Open+CompetitionDistance",data=Sales_Store_merged).fit()
print(model.summary(), sep='\n \n \n')

# The value of coefficient b3 of the independent variable named CompetitionDistance is positive but very small, ie 0.0296. The sign of the coefficient b3 of the independent 
# variable named CompetitionDistance, as well as its order of magnitude/value are not consistent with what we were expecting. Basically, the positive sign of the coefficient 
# b3 of the independent variable named CompetitionDistance does not make sense, since the we were expecting the higher the competition distance the lower the Sales would be,
# but the b3 coefficient suggests exactly the opposite, ie 1 unit increase in the value of the independent (predictor) variable named CompetitionDistance is expected to lead in
# an increase of 0.0296 ± 0.000 in the target variable Sales (in monetary (USD?) units). Also, the fact that the value of the coefficient b3 is small indicates that the 
# CompetitionDistance does not seem to have a big impact in the prediction of Sales, given the units that CompetitionDistance has in our model. The proper thing to do though,
# would be to normalize all the features of our model, and then run the regression model, so that the impact of the scales of the features do not lead us to misinterpretations 
# (would be a result of misleading values of the beta coefficients of our model), regarding the actual impact of the independent (predictor) variables to the dependent (target) 
# variable.

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.830
Model:                            OLS   Adj. R-squared:                  0.830
Method:                 Least Squares   F-statistic:                 1.647e+06
Date:                Tue, 24 Sep 2019   Prob (F-statistic):               0.00
Time:                        16:06:55   Log-Likelihood:            -8.9181e+06
No. Observations:             1014567   AIC:                         1.784e+07
Df Residuals:                 1014563   BIC:                         1.784e+07
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            -156.6383    